In [3]:
import pandas as pd # библиотека для удобной работы с датафреймами
import numpy as np # библиотека для удобной работы со списками и матрицами

# библиотека, где реализованы основные алгоритмы машинного обучения
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов"
#Импортируем токенизатор
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk
#скачать данные для nltk о пунктуации и стоп-словах
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kolcherindv2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kolcherindv2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# загружаем положительные твиты
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive) # устанавливаем метки

# загружаем отрицательные твиты
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative) # устанавливаем метки

# соединяем вместе
df = positive.append(negative)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label, random_state=42)

In [7]:
# инициализируем умный векторайзер 
from sklearn.feature_extraction.text import TfidfVectorizer
smart_Tfidfvectorizer = TfidfVectorizer(ngram_range=(1, 1), 
                                   tokenizer=word_tokenize)

In [8]:
# обучаем его и сразу применяем к x_train
smart_vectorized_x_train = smart_Tfidfvectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42, solver='newton-cg')
clf.fit(smart_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
smart_vectorized_x_test = smart_Tfidfvectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     28108
    positive       1.00      1.00      1.00     28601

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [17]:
import pickle
with open('Predskaz.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [18]:
with open('Vectorizer.pickle', 'wb') as f:
    pickle.dump(smart_Tfidfvectorizer, f)